In [3]:
import numpy as np
import statistics

In [4]:
# Exponential Ornstein-Uhlenbeck process

class XOU:

    def __init__(self, kappa, alpha, sigma, S0, r):

        self.kappa = kappa
        self.alpha = alpha
        self.sigma = sigma
        self.S0 = S0
        self.r = r

In [5]:
hw5dynamics=XOU(kappa = 0.472, alpha = 4.4, sigma = 0.368, S0 = 106.9, r = 0.05)

In [6]:
class CallOnForwardPrice:

    def __init__(self, K1, T1, T2):

        self.K1 = K1
        self.T1 = T1
        self.T2 = T2


In [7]:
hw5contract=CallOnForwardPrice(K1 = 103.2, T1 = 0.5, T2 = 0.75)

In [51]:
class MCengine:

    def __init__(self, N, M, epsilon, seed):

        self.N = N   # Number of timesteps on each path
        self.M = M   # Number of paths
        self.epsilon = epsilon  # For the dC/dS calculation
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
    
    
    def price_call_XOU(self, contract, dynamics):

        # You complete the coding of this function
        # self.rng.normal() generates pseudo-random normals
        
        self.rng.normal()
        # Risk-Neutral Dynamics Parameters
        K1, T1, T2 = contract.K1, contract.T1, contract.T2
        kappa, alpha, sigma, S0, r = dynamics.kappa, dynamics.alpha, dynamics.sigma, dynamics.S0, dynamics.r
        N, M, epsilon = self.N, self.M, self.epsilon
        deltat = T1 / N
        t = np.linspace(0, T1, num= N+1)
        S0_delta = S0 + epsilon
        
        
        St=[]
        Call_price=[]
        St_delta=[]
        Call_price_delta=[]
        Ft=[]
        Ft_delta=[]
        #simulate M paths by N steps from time 0 to T1
        for _ in range(M):
            # Make Brownian Motion
            Zt = np.random.normal(0, 1, size=N)
            Wt = np.sqrt(deltat) * Zt
            
            X0 = np.log(S0)
            X0_delta=np.log(S0_delta)
            Xt = [X0]
            Xt_delta=[X0_delta]
            # Iterate through all timesteps for the path
            for i in range(N):
            # Update next Xt position based on dynamics and store to list
                X_next = Xt[i] + (kappa * (alpha - Xt[i]) * deltat) + sigma * Wt[i]
                Xt.append(X_next)
            
                X_next_delta = Xt_delta[i] + (kappa * (alpha - Xt_delta[i]) * deltat) + sigma * Wt[i]
                Xt_delta.append(X_next_delta)
            
            
            St.append(np.exp(Xt[-1]))
            St_delta.append(np.exp(Xt_delta[-1]))
        
        for i in range(len(St)):
            
            time = T2 - T1 
            Ft_cal=np.exp( (np.exp(-kappa * time) * np.log(St[i])) +
                (alpha * (1 - np.exp(-kappa * time))) +
                (((sigma ** 2) / (4 * kappa)) * (1 - np.exp(-2 * kappa * time)))
                )
            Ft.append(Ft_cal)
            Ft_delta_cal=np.exp( (np.exp(-kappa * time) * np.log(St_delta[i])) +
                (alpha * (1 - np.exp(-kappa * time))) +
                (((sigma ** 2) / (4 * kappa)) * (1 - np.exp(-2 * kappa * time)))
                )
            Ft_delta.append(Ft_delta_cal)
            
            payoff=np.exp(-r*T1)*np.max([Ft_cal - K1,0])
            payoff_delta=np.exp(-r*T1)*np.max([Ft_delta_cal - K1,0])
            
            Call_price.append(payoff)
            Call_price_delta.append(payoff_delta)
        
        
        
        call_price=np.mean(Call_price)
        standard_error=np.std(Call_price)/np.sqrt(M)
        call_delta=(np.mean(Call_price_delta) - call_price) / epsilon
        
                
        return(call_price, standard_error, call_delta)


In [52]:
hw5MC = MCengine(N=100, M=100000, epsilon=0.01, seed=0)
# Change M if necessary

In [53]:
(call_price, standard_error, call_delta) = hw5MC.price_call_XOU(hw5contract,hw5dynamics)

In [54]:
print(call_price, standard_error, call_delta)

7.801332927608759 0.0420544686787495 0.34276938630322107


In [63]:
#Q2.b
from scipy.stats import norm
from scipy.optimize import brentq
import numpy as np

def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def implied_volatility(S, K, T, r, market_price):
    objective = lambda sigma: black_scholes_call(S, K, T, r, sigma) - market_price
    return brentq(objective, 0.01, 2.0)

# Market data
S = 100
K = 100
r = 0.05
expiries = [0.1, 0.2, 0.5]
market_prices = [5.25, 7.25, 9.5]

# Calculate implied volatilities
implied_vols = [implied_volatility(S, K, T, r, price) for T, price in zip(expiries, market_prices)]
implied_vols

[0.39732038579545226, 0.380171291551053, 0.2950972521755995]

In [64]:
#Q2.b
#t=0.4 vol=0.295
black_scholes_call(S, K, 0.4, r, 0.295)

8.391054915812

In [65]:
#t=0.1 vol=0.397
black_scholes_call(S, K, 0.1, r, 0.397)

5.24597934211392